**0. Mount gdrive! and check if GPU is allocated**

In [ ]:
# makesure NVIDIA GPU is enabled
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
#drive.flush_and_unmount() #do factory reset runtime if failed to load correct account
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls -l /mydrive

**1. Pull darknet from github**

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

**2. Create directory of opencv and enable GPU in darknet folders**

In [ ]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

**3. Load yolov3 cfgs files**

In [ ]:
# make copy for temp running trainning
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# configure darknet for yolov3
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create output folder
!mkdir "/mydrive/EdwardsLab"

In [ ]:
!echo "glycan" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/EdwardsLab' > data/obj.data
!mkdir data/obj

In [ ]:
# pulling initial weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

**4. Extract all uploaded image from glycan.zip**



In [ ]:
!unzip /mydrive/EdwardsLab/glycan.zip -d data/obj

In [ ]:
# import all png from object file
import glob
images_list = glob.glob("data/obj/*.png")
for name in images_list:
    print(name)

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**5. Begin the training from scratch with**

In [ ]:
# begin training (check if GPU enabled?) 

#starting new darknet53.conv.74
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# continue with existing training
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg /mydrive/yolov3/yolov3_training_last.weights -dont_show